In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from pathlib import Path

import pandas as pd
from loguru import logger

from multiplex_pipeline.core_cutting.channel_scanner import discover_channels
from multiplex_pipeline.core_cutting.controller import CorePreparationController
from multiplex_pipeline.core_cutting.file_io import LocalFileStrategy
from multiplex_pipeline.utils.config_loaders import load_analysis_settings

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [2]:
# load analysis configuration
settings_path = r'C:\BLCA-7_Analysis\analysis_settings_BLCA7.yaml'

settings = load_analysis_settings(settings_path)
settings

AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/BLCA-7/BLCA-7_Final', analysis_name='BLCA-7_Analysis', local_analysis_dir='C:/', remote_analysis_dir='/ix1/kkedziora/blca_analysis', log_dir=None), core_detection=CoreDetectionSettings(detection_image='BLCA-7_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=['008_ECad'], use_markers=['DAPI', 'CD45', 'CD44', 'HLA1', 'CD44', 'NaKATPase', 'CD11C', 'pCK26', 'pS6', 'HES1'], ignore_markers=['Antibody1', 'TNFa', 'Snail1', 'SKP2', 'ProgRc', 'Plk1', 'PH3', 'PDL1', 'p65', 'p130', 'p-p130', 'p-Cdc6', 'LAG3', 'IL-8', 'HER2', 'ERa', 'EpCAM', 'E2F1', 'cycD3', 'cycB2', 'CDC25C', 'CD86', 'CD73', 'CD69', 'CD62L', 'CD56', 'CD4', 'CD25', 'CD19', 'CD27', 'CCR7', 'cCASP3'], margin=0, mask_value=0, transfer_cleanup_enabled=True, cor

### Define the logger

In [3]:
log_file = settings.log_dir_path / f"cores_cutting_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

### Define cores for processing

In [4]:
df_path = settings.core_info_file_path.with_suffix('.pkl')

df = pd.read_pickle(df_path)
df.head()

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,2304.0,7296.0,8000.0,13312.0,rectangle,"[[2304.0, 13312.0], [7296.0, 13312.0], [7296.0..."
1,Core_001,1984.0,7232.0,14400.0,19840.0,rectangle,"[[1984.0, 19840.0], [7232.0, 19840.0], [7232.0..."
2,Core_002,2304.0,7744.0,21952.0,26048.0,rectangle,"[[2304.0, 26048.0], [7744.0, 26048.0], [7744.0..."
3,Core_003,2496.0,7808.0,27200.0,32768.0,rectangle,"[[2496.0, 32768.0], [7808.0, 32768.0], [7808.0..."
4,Core_004,2560.0,8000.0,34048.0,39488.0,rectangle,"[[2560.0, 39488.0], [8000.0, 39488.0], [8000.0..."


In [5]:
# create a subset of cores (optional)
df = df[:1]
df

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,2304.0,7296.0,8000.0,13312.0,rectangle,"[[2304.0, 13312.0], [7296.0, 13312.0], [7296.0..."


## Local files

In [6]:
channel_map = discover_channels(Path(settings.general.image_dir),
                                include_channels=settings.core_cutting.include_channels,
                                exclude_channels=settings.core_cutting.exclude_channels,
                                use_markers=settings.core_cutting.use_markers,
                                ignore_markers=settings.core_cutting.ignore_markers)

2025-10-17 13:30:57.022 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:80 - Discovered 131 channels:
2025-10-17 13:30:57.027 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 001_CDC25C <- R:\CellDive\BLCA-7\BLCA-7_Final\BLCA-7_1.0.4_R000_FITC_CDC25C-AF488_FINAL_AFR_F.ome.tif
2025-10-17 13:30:57.028 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 001_DAPI <- R:\CellDive\BLCA-7\BLCA-7_Final\BLCA-7_1.0.4_R000_DAPI__FINAL_F.ome.tif
2025-10-17 13:30:57.028 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 001_cycD3 <- R:\CellDive\BLCA-7\BLCA-7_Final\BLCA-7_1.0.4_R000_Cy7_cycD3-AF750_FINAL_AFR_F.ome.tif
2025-10-17 13:30:57.029 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 001_pH2AX <- R:\CellDive\BLCA-7\BLCA-7_Final\BLCA-7_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F.ome.tif
2025-10-17 13:30:57.0

In [ ]:
# # for tests you can request a small set of channels here
# # then send 'short_map' to the controller instead of 'channel_map'
# selected_keys = ["DAPI", "CD3"]
# short_map = {k: channel_map[k] for k in selected_keys if k in channel_map}
# short_map

{'DAPI': 'R:\\CellDive\\BLCA-1B\\BLCA-1B_Final\\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'CD3': 'R:\\CellDive\\BLCA-1B\\BLCA-1B_Final\\BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif'}

In [7]:
strategy = LocalFileStrategy()

controller = CorePreparationController(
    metadata_df = df, # df defines which cores to process
    image_paths = channel_map, # defines which channels to use
    temp_dir = settings.cores_dir_tif_path,
    output_dir = settings.cores_dir_output_path,
    file_strategy = strategy,
    margin = settings.core_cutting.margin,
    mask_value = settings.core_cutting.mask_value,
    max_pyramid_levels = settings.sdata_storage.max_pyramid_level,
    chunk_size = settings.sdata_storage.chunk_size,
    downscale = settings.sdata_storage.chunk_size,
)

controller.run()

2025-10-17 13:30:59.925 | INFO     | multiplex_pipeline.core_cutting.controller:run:117 - Starting controller run loop...
2025-10-17 13:31:02.439 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:93 - Cut and saved core Core_000, channel DAPI
2025-10-17 13:31:02.453 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:100 - Closed file handle for channel DAPI
2025-10-17 13:31:04.376 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:93 - Cut and saved core Core_000, channel HES1
2025-10-17 13:31:04.399 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:100 - Closed file handle for channel HES1
2025-10-17 13:31:06.370 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:93 - Cut and saved core Core_000, channel pCK26
2025-10-17 13:31:06.397 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:100 - Closed file handle for channel pCK26
2025-10-17 13:31:08.208 | DEBUG    | multiplex_pipeline.c